In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, date_format, concat_ws, to_timestamp, to_date, expr, StringType, collect_list
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("FB").getOrCreate()

df = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0.csv", header=True, inferSchema=True)
df2 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (1).csv", header=True, inferSchema=True)
df3 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (2).csv", header=True, inferSchema=True)
df4 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (3).csv", header=True, inferSchema=True)


In [34]:
df.show(5)

+---+----------+-------------------+-----------+-------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+----+----+----+----+----+----+----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|   HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|  Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA| PSH| PSD|  PSA| WHH|WHD| WHA| VCH| VCD| VCA|MaxH|MaxD|MaxA|AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|Max<2.5|Avg>2.5|Avg<2.5|

In [35]:
df = df.union(df2)
df = df.union(df3)
df = df.union(df4)

In [36]:
df = df.drop("Div")
df = df.select("Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR")

In [37]:
dfHome = df.select(col("Time"), col("HomeTeam").alias("Team"), col("FTR"))
dfAway = df.select(col("Time"), col("AwayTeam").alias("Team"), col("FTR"))

dfForm = dfHome.union(dfAway).orderBy("Time")
dfForm = dfForm.select("Time", "Team", "FTR")

#Last 5 game results for the home team
window_spec = Window.partitionBy("Team").orderBy("Time").rowsBetween(Window.unboundedPreceding, Window.currentRow)

dfForm = dfForm.withColumn("results_list", collect_list("FTR").over(window_spec))

dfForm.createOrReplaceTempView("games")
query = """
SELECT *,
    CASE 
        WHEN size(results_list) >= 5 THEN concat_ws('', slice(results_list, size(results_list) - 4, 5))
        ELSE concat_ws('', results_list)
    END AS TeamForm
FROM games
"""

dfForm = spark.sql(query)
dfForm = dfForm.drop("results_list")

home_form_df = dfForm.withColumnRenamed("Team", "HomeTeam").withColumnRenamed("TeamForm", "HomeTeamForm")
away_form_df = dfForm.withColumnRenamed("Team", "AwayTeam").withColumnRenamed("TeamForm", "AwayTeamForm")

# Join on HomeTeam and Time
df = df.join(home_form_df, on=[df.HomeTeam == home_form_df.HomeTeam, df.Time == home_form_df.Time], how="left").drop(home_form_df.HomeTeam).drop(home_form_df.Time)

# Join on AwayTeam and Time
df = df.join(away_form_df, on=[df.AwayTeam == away_form_df.AwayTeam, df.Time == away_form_df.Time], how="left").drop(away_form_df.AwayTeam).drop(away_form_df.Time)

df.show()

+----------+--------+--------+----+----+---+----+----+---+--------+---+---+---+---+---+---+---+---+---+---+---+---+-------------------+---+------------+-------------------+---+------------+
|      Date|HomeTeam|AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR| Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|               Time|FTR|HomeTeamForm|               Time|FTR|AwayTeamForm|
+----------+--------+--------+----+----+---+----+----+---+--------+---+---+---+---+---+---+---+---+---+---+---+---+-------------------+---+------------+-------------------+---+------------+
|11/08/2023| Burnley|Man City|   0|   3|  A|   0|   2|  A|C Pawson|  6| 17|  1|  8| 11|  8|  6|  5|  0|  0|  1|  0|2024-07-24 20:00:00|  A|       HDHAA|2024-07-24 20:00:00|  A|       ADDAA|
|11/08/2023| Burnley|Man City|   0|   3|  A|   0|   2|  A|C Pawson|  6| 17|  1|  8| 11|  8|  6|  5|  0|  0|  1|  0|2024-07-24 20:00:00|  A|       HDHAA|2024-07-24 20:00:00|  A|       AADDA|
|11/08/2023| Burnley|Man City|   0|   3|  A|   0| 

In [38]:
df.groupBy("Referee").count().orderBy("count", ascending=False).show(300)

+------------+-----+
|     Referee|count|
+------------+-----+
|    S Hooper|57191|
|   P Tierney|56150|
|    A Taylor|54159|
|    M Oliver|46730|
|    A Madley|45844|
|    C Pawson|45093|
|    P Bankes|41125|
|     D Coote|38251|
|  C Kavanagh|38040|
|   S Attwell|36356|
|   D England|35597|
| M Salisbury|35520|
|     R Jones|34597|
|     G Scott|34155|
|    J Brooks|29212|
|   J Gillett|26603|
|  A Marriner|26408|
|      J Moss|26394|
|   T Bramall|19304|
|  M Atkinson|18694|
|  T Robinson|17752|
|    K Friend|17262|
|   S Barrott|16074|
|      M Dean|16036|
|T Harrington|11364|
|      D Bond| 9960|
|     J Smith| 6431|
|    R Madley| 4753|
|     L Smith| 4572|
|     L Mason| 2229|
|   S Allison| 2116|
|   M Donohue| 2060|
|     R Welch| 2054|
|     S Singh| 1320|
|  J Gillett |  725|
|      A Moss|    6|
+------------+-----+



In [39]:
df = df.withColumn("corrected_date", expr("concat('20', substr(date, 9, 2), '-', substr(date, 4, 2), '-', substr(date, 1, 2))"))
df = df.withColumn("corrected_date_type", to_date(col("corrected_date")))
df = df.withColumn("updated_datetime", (to_timestamp(concat_ws(' ', date_format(col("corrected_date_type"), 'yyyy-MM-dd'), date_format(col("Time"), 'HH:mm:ss')))))
df = df.drop("Date", "Time", "corrected_date")
df = df.withColumnRenamed("corrected_date_type", "Date")
df = df.withColumnRenamed("updated_datetime", "Time")


AnalysisException: [AMBIGUOUS_REFERENCE] Reference `Time` is ambiguous, could be: [`games`.`Time`, `games`.`Time`].

In [ ]:
df.head(5)

[Row(HomeTeam='Burnley', AwayTeam='Man City', FTHG=0, FTAG=3, FTR='A', HTHG=0, HTAG=2, HTR='A', Referee='C Pawson', HS=6, AS=17, HST=1, AST=8, HF=11, AF=8, HC=6, AC=5, HY=0, AY=0, HR=1, AR=0, Date=datetime.date(2023, 8, 11), Time=datetime.datetime(2023, 8, 11, 20, 0)),
 Row(HomeTeam='Arsenal', AwayTeam="Nott'm Forest", FTHG=2, FTAG=1, FTR='H', HTHG=2, HTAG=0, HTR='H', Referee='M Oliver', HS=15, AS=6, HST=7, AST=2, HF=12, AF=12, HC=8, AC=3, HY=2, AY=2, HR=0, AR=0, Date=datetime.date(2023, 8, 12), Time=datetime.datetime(2023, 8, 12, 12, 30)),
 Row(HomeTeam='Bournemouth', AwayTeam='West Ham', FTHG=1, FTAG=1, FTR='D', HTHG=0, HTAG=0, HTR='D', Referee='P Bankes', HS=14, AS=16, HST=5, AST=3, HF=9, AF=14, HC=10, AC=4, HY=1, AY=4, HR=0, AR=0, Date=datetime.date(2023, 8, 12), Time=datetime.datetime(2023, 8, 12, 15, 0)),
 Row(HomeTeam='Brighton', AwayTeam='Luton', FTHG=4, FTAG=1, FTR='H', HTHG=1, HTAG=0, HTR='H', Referee='D Coote', HS=27, AS=9, HST=12, AST=3, HF=11, AF=12, HC=6, AC=7, HY=2, AY=2